**Markov Decision Process**

A Markov decision process (MDP), by definition, is a sequential decision problem for a fully observable, stochastic environment with a Markovian transition model and additive rewards. It consists of a set of states, a set of actions, a transition model, and a reward function. MDP usually has a discount factor γ , a number between 0 and 1, that describes the preference of an agent for current rewards over future rewards.


**1)Value Iteration and Policy Iteration**

**Value iteration** is an algorithm that gives an optimal policy for a MDP. It calculates the utility of each state, which is defined as the expected sum of discounted rewards(called the Bellman equation) from that state onward. For n states, there are n Bellman equations with n unknowns (the utilities of states). To solve this system of equations, value iteration uses an iterative approach that repeatedly updates the utility of each state (starting from zero) until an equilibrium is reached (converge). 
Here's the pseudocode for calculating the utilities of states.

Then, after the utilities of states are calculated, we can use them to select an optimal action for each state.


**Policy iteration** is another algorithm that solves MDPs. It starts with a random policy and alternates the following two steps until the policy improvement step yields no change:

(1) Policy evaluation: given a policy, calculate the utility U(s) of each state s if the policy is executed;

(2) Policy improvement: update the policy based on U(s).

For the policy evaluation step, we use a simplified version of the Bellman equation to calculate the utility of each state.

For n states, there are n linear equations with n unknowns (the utilities of states), which can be solved in O(n^3) time. To make the algorithm more efficient, we can perform some number of simplified Bellman updates (simplified because the policy is fixed) to get an approximation of the utilities instead of calculating the exact solutions.

In [36]:
import numpy as np
'''==================================================
GridWorldMDP class
=================================================='''
class GridWorldMDP:

    def __init__(self, reward_grid, goal_state, actions, alpha=0.5, epsilon=0.5, epsilon_decay=0.99 , gamma=0.9, theta=0.005, itrs=100, episodes=400, episode_length=100):
        self._reward_grid = reward_grid
        self._goal_state = goal_state
        self._actions = actions
        self._alpha = alpha
        self._epsilon = epsilon
        self._epsilon_decay = epsilon_decay
        self._gamma = gamma
        self._theta = theta
        self._iterations = itrs
        self._episodes = episodes
        self._episode_length = episode_length
        self._value = np.zeros(reward_grid.shape, float)
        self._policy = np.zeros(reward_grid.shape, int)
        self._qsa_grid = np.zeros([reward_grid.shape[0], reward_grid.shape[1], len(self._actions)], float)
        self._transition = np.ones([reward_grid.shape[0], reward_grid.shape[1], len(self._actions)], int)*-1
        #in the middel section it can take all four directions
        for i in range(1,reward_grid.shape[0]-1):
            for j in range(1,reward_grid.shape[1]-1):
                for a in range(len(self._actions)):
                    new_x = self._actions[a][0] + i
                    new_y = self._actions[a][1] + j
                    self._transition[i][j][a] = new_x*self._reward_grid.shape[1] + new_y

        #Left most side
        for i in range(1,reward_grid.shape[0]-1):
            for a in [0,1,2]:
                new_x = self._actions[a][0] + i
                new_y = self._actions[a][1] + 0
                self._transition[i][0][a] = new_x*self._reward_grid.shape[1] + new_y

        #Right most side
        for i in range(1,reward_grid.shape[0]-1):
            for a in [0,2,3]:
                new_x = self._actions[a][0] + i
                new_y = self._actions[a][1] + self._reward_grid.shape[1]-1
                self._transition[i][self._reward_grid.shape[1]-1][a] = new_x*self._reward_grid.shape[1] + new_y

        #Top side
        for j in range(1,reward_grid.shape[1]-1):
            for a in [1,2,3]:
                new_x = self._actions[a][0] + 0
                new_y = self._actions[a][1] + j
                self._transition[0][j][a] = new_x*self._reward_grid.shape[1] + new_y

        #Bottom side
        for j in range(1,reward_grid.shape[1]-1):
            for a in [0,1,3]:
                new_x = self._actions[a][0] + self._reward_grid.shape[0]-1
                new_y = self._actions[a][1] + j
                self._transition[self._reward_grid.shape[0]-1][j][a] = new_x*self._reward_grid.shape[1] + new_y

        #Left top corner
        new_x = self._actions[1][0] + 0
        new_y = self._actions[1][1] + 0
        self._transition[0][0][1] = new_x*self._reward_grid.shape[1] + new_y #Right
        new_x = self._actions[2][0] + 0
        new_y = self._actions[2][1] + 0
        self._transition[0][0][2] = new_x*self._reward_grid.shape[1] + new_y #Down

        #Right top corner
        new_x = self._actions[2][0] + 0
        new_y = self._actions[2][1] + self._reward_grid.shape[1]-1
        self._transition[0][self._reward_grid.shape[1]-1][2] = new_x*self._reward_grid.shape[1] + new_y #Down
        new_x = self._actions[3][0] + 0
        new_y = self._actions[3][1] + self._reward_grid.shape[1]-1
        self._transition[0][self._reward_grid.shape[1]-1][3] = new_x*self._reward_grid.shape[1] + new_y #Left

        #Right bottom corner
        new_x = self._actions[0][0] + self._reward_grid.shape[0]-1
        new_y = self._actions[0][1] + self._reward_grid.shape[1]-1
        self._transition[self._reward_grid.shape[0]-1][self._reward_grid.shape[1]-1][0] = new_x*self._reward_grid.shape[1] + new_y #UP
        new_x = self._actions[3][0] + self._reward_grid.shape[0]-1
        new_y = self._actions[3][1] + self._reward_grid.shape[1]-1
        self._transition[self._reward_grid.shape[0]-1][self._reward_grid.shape[1]-1][3] = new_x*self._reward_grid.shape[1] + new_y #Left

        #Left Bottom corner
        new_x = self._actions[0][0] + self._reward_grid.shape[0]-1
        new_y = self._actions[0][1] + 0
        self._transition[self._reward_grid.shape[0]-1][0][0] = new_x*self._reward_grid.shape[1] + new_y #UP
        new_x = self._actions[1][0] + self._reward_grid.shape[0]-1
        new_y = self._actions[1][1] + 0
        self._transition[self._reward_grid.shape[0]-1][0][1] = new_x*self._reward_grid.shape[1] + new_y #Right
        
    def one_step_lookahead_all(self, state, V): #all posible next-values or 
        A = {}
        for a in range(len(self._actions)):          
            s = self._transition[state[0]][state[1]][a]
            if s > -1: #If Action is allowed
                A[a] = self._reward_grid[state[0]][state[1]] + self._gamma * V[s//self._reward_grid.shape[1]][s%self._reward_grid.shape[1]]
        return A
    
    def value_iteration(self):
        print("*******######## VALUE ITERATION ########*******\n")
        itr=0
        V = np.zeros(self._reward_grid.shape, float)
        while True:
            # Stopping condition
            delta = 0
            # Update each state...
            for i in range(self._reward_grid.shape[0]):
                for j in range(self._reward_grid.shape[1]):
                    if (i,j) != self._goal_state:
                        # Do a one-step lookahead to find the best action
                        A = self.one_step_lookahead_all((i,j),V)
                        best_action_value = max(A.values())
                        # Calculate delta across all states seen so far
                        delta = max(delta, np.abs(best_action_value - V[i][j]))
                        # Update the value function. 
                        V[i][j] = best_action_value
            # Check if we can stop
            if (delta < self._theta) or (itr > self._iterations):
                break
            # print progress
            self._value = V
            print("Value iteration:{}  V = \n".format(itr))
            self.printEnvironment(False)
            itr += 1
        # calculate the policy using the optimal value function found
        for i in range(self._reward_grid.shape[0]):
            for j in range(self._reward_grid.shape[1]):
                #skip terminal states
                if (i == self._goal_state[0] and j == self._goal_state[1]):
                    continue
                # One step lookahead to find the best action for this state
                A = self.one_step_lookahead_all((i,j), self._value)
                best_action = max(A, key= lambda x: A[x])
                # Always take the best action
                self._policy[i][j] = best_action
        return self._policy, self._value
        
    def policyEvaluation(self, P, V): #Value iterate on a given policy and starting value untill convergence
        while True:
            nextV = np.zeros(self._reward_grid.shape, float)
            delta = 0
            for i in range(self._reward_grid.shape[0]):
                for j in range(self._reward_grid.shape[1]):
                    if (i,j) != self._goal_state:
                        s = self._transition[i][j][P[i][j]]
                        if s > -1:#If Action(policy) is allowed
                            nextV[i][j] = self._reward_grid[i][j] + self._gamma * V[s//self._reward_grid.shape[1]][s%self._reward_grid.shape[1]]
                            delta = max(delta, abs(nextV[i][j]-V[i][j]))
            V = nextV
            if delta < self._theta:
                break
        return V
    
    def policy_iteration(self):
        print("*******######## POLICY ITERATION ########*******\n")
        #Initialize random policy
        P = np.array([[np.random.randint(0, 3) for j in range(self._reward_grid.shape[1])] for i in range(self._reward_grid.shape[0])])
        V = np.zeros(self._reward_grid.shape, float)
        itr=0
        while True:
            V = self.policyEvaluation(P,V)
            unchanged = True
            for i in range(self._reward_grid.shape[0]):
                for j in range(self._reward_grid.shape[1]):
                    if (i,j) != self._goal_state:
                        #find the best action from current state
                        maxAction, maxV , cur_v = None, -float("inf"), -float("inf")
                        for a in range(len(self._actions)):
                            s = self._transition[i][j][a]
                            if s > -1:#If Action(policy) is allowed
                                v = self._reward_grid[i][j] + self._gamma * V[s//self._reward_grid.shape[1]][s%self._reward_grid.shape[1]]
                                if v > maxV:
                                    maxAction, maxV = a, v
                        #find the value of the current action in the current policy
                        s = self._transition[i][j][P[i][j]]
                        if s > -1:#If Action(of current policy) NOT allowed, use the serached maxAction
                            cur_v = self._reward_grid[i][j] + self._gamma * V[s//self._reward_grid.shape[1]][s%self._reward_grid.shape[1]]
                        if maxV > cur_v:
                            P[i][j] = maxAction # the action that maximizes the utility
                            unchanged = False
            # Check if we can stop
            if unchanged or (itr > self._iterations):
                break
            # print progress
            self._policy = P
            print("Policy iteration:{}  P = \n".format(itr))
            self.printEnvironment(True)
            itr += 1
        self._value = V
        return P, V
    def printEnvironment(self,policy=False):
        res = ""
        for i in range(self._reward_grid.shape[0]):
            res += "|"
            for j in range(self._reward_grid.shape[1]):
                if (i, j) == self._goal_state:
                    val = "+10"
                else:
                    if policy:
                        val = ["Up", "Right","Down", "Left"][self._policy[i][j]]
                    else:
                        val = str(self._value[i][j])
                res += " " + val[:5].ljust(5) + " |" # format
            res += "\n"
        print(res)
    def print_q(self):
        print(" State |   Up  |  Right  |  Down   |  Left\n")
        for i in range(self._reward_grid.shape[0]):
            for j in range(self._reward_grid.shape[1]):
                print(" ({},{}) | {:.5f} | {:.5f} | {:.5f} | {:.5f} \n".format(i,j,self._qsa_grid[i][j][0],self._qsa_grid[i][j][1],self._qsa_grid[i][j][2],self._qsa_grid[i][j][3]))
    def get_next_state(self, qsa_grid, x, y):
        #choose b/n the optimal action from Q(Exploitation) and a random action(Exploration) 
        if np.random.uniform(0,1) > self._epsilon:
            action = np.argmax(qsa_grid[x][y])
        else:
            action = np.random.randint(0, len(self._actions))
        xf, yf = x, y
        s = self._transition[x][y][action]
        if s > -1: # if action is allowed, update to new state
            xf = s // self._reward_grid.shape[1]
            yf = s % self._reward_grid.shape[1]
        
        r = self._reward_grid[xf][yf]
        return (xf, yf, r, action)

    def run_one_episode(self, qsa_grid):
        x, y = 0, 0
        while (x, y) == self._goal_state:
            x, y = np.random.randint(0, self._reward_grid.shape[0]), np.random.randint(0, self._reward_grid.shape[1])
        reward_per_episode = 0
        for ep_len in range(episode_length):
            xf, yf, r, action = self.get_next_state(qsa_grid, x, y)
            qsa_grid[x][y][action] += self._alpha*(r + self._gamma*np.max(qsa_grid[xf][yf]) - qsa_grid[x][y][action])
            reward_per_episode += r
            x, y = xf, yf
            if (xf, yf) == self._goal_state:
                break
        self._epsilon *= self._epsilon_decay
        return qsa_grid, reward_per_episode

    def q_learning(self):
        print("*******######## Q-learning ########*******\n")
        qsa_grid = self._qsa_grid
        rewards_per_episode = []
        for eps in range(episodes):
            qsa_grid, r = self.run_one_episode(qsa_grid)
            rewards_per_episode.append(r)
            print("Episode {} got reward of {:.5f} and the Q-table:\n".format(eps+1,r))
            self.print_q()

        P = np.zeros(self._reward_grid.shape, int)
        V = np.zeros(self._reward_grid.shape, float)
        for x in range(self._reward_grid.shape[0]):
            for y in range(self._reward_grid.shape[1]):
                P[x][y] = np.argmax(qsa_grid[x][y])
                V[x][y] = np.max(qsa_grid[x][y])
        self._value = V
        self._policy = P
        return V, P, rewards_per_episode


In [41]:
#creat and initialize world
h, l = 5,5 #world dimension
goal_state = (3,2) #goal state
reward_grid = np.ones([h,l])*-1 #reward for non-goal states
reward_grid[goal_state[0]][goal_state[1]] = 10 #reward for goal state
print(reward_grid)
actions = [(-1, 0),(0, 1),(1, 0),(0, -1)]
gamma = 0.9 #discount factor
theta = 0.0005 #error threshold
itrs = 100 #maximum no of value iterations

[[-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. 10. -1. -1.]
 [-1. -1. -1. -1. -1.]]


In [ ]:
#creat and initialize world with user input
h,l = [int(x) for x in input('Please enter grid size(height and length separated by space like \"5 5\":').split()]
r_grid = np.ones([h,l])*-1
while True:
  g = [int(x) for x in input('Specify the GOAL state on this {}X{} grid:'.format(h,l)).split()]
  if((g[0] >= h) or (g[0] < 0) or (g[1] < 0) or (g[1] >= l)):
    print("*******Goal is NOT inside grid******\n")
  else:
    break
r_grid[g[0]][g[1]] = 10
while True:
  gama = float(input('Specify the discount factor(between 0 and 1):'))
  if gama <= 0.0 or gama >=1.0:
    print("*******choose gamma in the range(0,1)******\n")
  else:
    break
while True:
  theta = float(input('Specify the error threshold(less than 0.5):'))
  if theta <= 0.0 or theta >=0.5:
    print("*******choose theta in the range(0,0.5)******\n")
  else:
    break
while True:
  itr = int(input('Specify maximum number of iterations:'))
  if itr <= 0:
    print("*******choose a +ve integer******\n")
  else:
    break

In [33]:
value_GW = GridWorldMDP(reward_grid, goal_state, actions, alpha, epsilon, epsilon_decay, gamma, theta, itrs, episodes, episode_length)
print("Initial Policy:\n")
value_GW.printEnvironment(True)
print("\nInitial Value:\n")
value_GW.printEnvironment(False)

P,V = value_GW.value_iteration()
value_GW.printEnvironment(True)

Initial Policy:

| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | +10   | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |


Initial Value:

| 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   |
| 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0  

In [34]:
policy_GW = GridWorldMDP(reward_grid, goal_state, actions, alpha, epsilon, epsilon_decay, gamma, theta, itrs, episodes, episode_length)
print("Initial Pplicy\n")
policy_GW.printEnvironment(True)
print("\nInitial Value\n")
policy_GW.printEnvironment(False)

P,V = policy_GW.policy_iteration()
policy_GW.printEnvironment(False)

Initial Pplicy

| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | +10   | Up    | Up    | Up    | Up    |
| Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    | Up    |


Initial Value

| 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   |
| 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   | 0.0   |

In [42]:
alpha=0.5 #learning rate
epsilon=0.5 #exploartion vs exploitation
epsilon_decay=1 
episodes=100 #total number of episodes to experience
episode_length=100 # max length of an episode
Q_GW = GridWorldMDP(reward_grid, goal_state, actions, alpha, epsilon, epsilon_decay, gamma, theta, itrs, episodes, episode_length)

P,V,R= Q_GW.q_learning()
print("Reward seqwence:\n{}".format(R))
print("Final Pplicy:\n")
Q_GW.printEnvironment(True)
print("Final Value:\n")
Q_GW.printEnvironment(False)

*******######## Q-learning ########*******

Episode 1 got reward of -100.00000 and the Q-table:

 State |   Up  |  Right  |  Down   |  Left

 (0,0) | -0.50000 | -0.50000 | -0.50000 | -0.87500 

 (0,1) | -0.75000 | -0.50000 | -0.87500 | -0.50000 

 (0,2) | -0.75000 | -1.08750 | -0.75000 | -0.72500 

 (0,3) | -1.50312 | -1.38125 | -1.10000 | -0.97500 

 (0,4) | -1.21250 | -0.97500 | -1.16250 | -1.31375 

 (1,0) | 0.00000 | -0.75000 | 0.00000 | -0.50000 

 (1,1) | -0.87500 | -0.50000 | -0.50000 | -0.50000 

 (1,2) | -0.97500 | -1.21250 | -0.50000 | 0.00000 

 (1,3) | -1.21250 | -0.97500 | -0.75000 | -0.75000 

 (1,4) | -1.44375 | -0.75000 | -0.97500 | -0.97500 

 (2,0) | 0.00000 | 0.00000 | 0.00000 | 0.00000 

 (2,1) | -0.50000 | 0.00000 | 0.00000 | 0.00000 

 (2,2) | -0.50000 | -0.50000 | 0.00000 | 0.00000 

 (2,3) | -0.50000 | -0.72500 | -0.75000 | -0.50000 

 (2,4) | -0.97500 | -0.75000 | -0.50000 | -0.50000 

 (3,0) | 0.00000 | 0.00000 | 0.00000 | 0.00000 

 (3,1) | 0.00000 | 0.00000 